In [85]:
import os
import pandas as pd
import numpy as np

from module_dossard import MiseEnGrille
from module_ranking import Ranking

from utils import clean_club, get_UCD_top

In [75]:
path_inscrits = '/Users/claroche/Downloads/1ere-Manche-Challenge-Darnetalais-2023-2023-01-26-2.xlsx'
path_ref = '/Users/claroche/Downloads/general darnetal.xlsx'
path2em = r'/Users/claroche/Downloads/2eme-Manche-Challenge-Darnetalais-2022-export-2023-01-25.xlsx'
path3em = r'/Users/claroche/Downloads/3eme-Manche-Challenge-Darnetalais-2022-export-2023-01-25.xlsx'

In [76]:
grille = pd.read_excel(path_ref, sheet_name='grille de points')
grille = list(grille['Pts Scratch'])
for i in range(300):
    grille.append(0)

In [86]:
def clean_excel(string):
    return string.replace('/','_')

In [82]:
save_path = '/Users/claroche/Desktop/test_mise_grille.xlsx'

# Open inscrits
categories = ['7/8 ans', '9/10 ans', '11/12 ans', '13/14 ans', '15/16 ans', 'Scratch']
inscrits = {}

jeunes_9_16 = pd.read_excel(path_inscrits, sheet_name="JEUNES 9- 16 ans HF")

for i in categories:
    if i == "Scratch":
        inscrits[i] = pd.read_excel(path_inscrits, sheet_name="ADULTES HF")
    elif i == "7/8 ans":
        inscrits["7/8 ans"] = pd.read_excel(path_inscrits, sheet_name="JEUNES 7-8 ans HF")
    else:
        inscrits[i] = jeunes_9_16[jeunes_9_16['Catégorie'] == i]

# Get ranking
name = ['Chrono - 7 8 ans', 'Chrono - 9 10 ans', 'Chrono - 11 12 ans', 'Chrono - 13 14 ans', 'Chrono - 15 16 ans', 'Chrono']
references = {}
for i,j in zip(name, categories):
    ranking = Ranking(grille)
    ranking.add_first_ranking(pd.read_excel(path2em, sheet_name=i))
    ranking.add_ranking(pd.read_excel(path3em, sheet_name=i))
    ref = ranking.return_rank()
    ref['Display name'] = ref['Nom affiché'].apply(clean_club)
    references[j] = ref

dossards = {}
with pd.ExcelWriter(save_path) as writer: 
    for i in categories:
        meg = MiseEnGrille(inscrits[i], references[i])
        if i == 'Scratch':
            dossards[i] = meg.get_dossard()
            dossards[i].to_excel(writer, sheet_name=clean_excel(i))
        else:
            dossards[i] = get_UCD_top(meg.get_dossard())
            dossards[i].to_excel(writer, sheet_name=clean_excel(i))

/Users/claroche/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/claroche/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/claroche/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/claroche/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/claroche/opt/anaconda3/lib/python3.9/site

In [19]:
import argparse
import os
import pandas as pd

from utils import clean_excel, clean_club, get_UCD_top

from module_dossard import MiseEnGrille
from module_ranking import Ranking

grille = [100, 95, 90, 86, 82, 79, 76, 73, 70, 68, 66, 64, 62, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50,
          49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25,
          24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
grille += [0 for i in range(300)]

important_col = ['First name', 'Last name', 'Sexe', 'Date de Naissance', 'Catégorie', 'Licence Type', 'Club Sportif']


opt = {'ref_folder': 'results_challenge_2023', 'inscrits_path': '/Users/claroche/Downloads/2nd-manche-du-Challenge-Darnetalais-2023-2023-02-20.xlsx'}

print('-' * 50)
print('Dossard à partir du fichier ' + opt['inscrits_path'] + ' en préparation...')
print('-' * 50)

ref_list = [os.path.join(opt['ref_folder'], i) for i in os.listdir(opt['ref_folder']) if i.endswith('.xlsx')]

# Open inscrits
categories = ['7/8 ans', '9/10 ans', '11/12 ans', '13/14 ans', '15/16 ans', 'Scratch']
inscrits = {}

jeunes_9_16 = pd.read_excel(opt['inscrits_path'], sheet_name="JEUNES 9- 16 ans HF")

for i in categories:
    if i == "Scratch":
        inscrits[i] = pd.read_excel(opt['inscrits_path'], sheet_name="ADULTES HF")
    elif i == "7/8 ans":
        inscrits["7/8 ans"] = pd.read_excel(opt['inscrits_path'], sheet_name="JEUNES 7-8 ans HF")
    else:
        inscrits[i] = jeunes_9_16[jeunes_9_16['Catégorie'] == i]
# Get ranking
name = ['Chrono - 7 8 ans', 'Chrono - 9 10 ans', 'Chrono - 11 12 ans', 'Chrono - 13 14 ans', 'Chrono - 15 16 ans', 'Chrono']
references = {}
for i,j in zip(name, categories):
    ranking = Ranking(grille)
    ranking.add_first_ranking(pd.read_excel(ref_list[0], sheet_name=i))
    for k in range(1, len(ref_list)):
        ranking.add_ranking(pd.read_excel(ref_list[k], sheet_name=i))

    ref = ranking.return_rank()
    ref['Display name'] = ref['Nom affiché'].apply(clean_club)
    references[j] = ref

print(references['Scratch'])

--------------------------------------------------
Dossard à partir du fichier /Users/claroche/Downloads/2nd-manche-du-Challenge-Darnetalais-2023-2023-02-20.xlsx en préparation...
--------------------------------------------------


/Users/claroche/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/claroche/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/claroche/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/claroche/Desktop/code_misegrille/module_ranking.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

     Classement        Nom affiché    Categorie   Sexe           Club  tot  \
0             1        Theo AUZOUX  20/29 ans H  HOMME            NaN  100   
1             2    Nicolas RIHOUEY  20/29 ans H  HOMME    Uc darnétal   95   
2             3  Axel HARDY--LAINÉ  17/19 ans H  HOMME    UC Darnetal   90   
3             4    Arthur TROPARDY  30/39 ans H  HOMME       VC Rouen   86   
4             5   Damien GUILLEMET  30/39 ans H  HOMME    XC Couronne   82   
..          ...                ...          ...    ...            ...  ...   
319         320    Elodie VERHAEGE  30/39 ans F  FEMME   Tac cyclisme    0   
320         321   Valentin  BREANT  17/19 ans H  HOMME            NaN    0   
321         322     Batiste  EMERY    13/14 ans  HOMME         vcpacy    0   
322         323     Johan FONTAINE  50/59 ans H  HOMME  AC SOTTEVILLE    0   
323         324     Tom CARTHALADE    15/16 ans  HOMME     ucdarnetal    0   

        Display name  
0         theoauzoux  
1     nicolasriho

/Users/claroche/Desktop/code_misegrille/module_ranking.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.rank['tot'] = self.rank['Classement'].apply(lambda x: rank2point(x, self.grille))
/var/folders/ww/4jcvd4mx75v0qwx2q38mr4s00000gn/T/ipykernel_24159/1055785284.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ref['Display name'] = ref['Nom affiché'].apply(clean_club)


In [29]:
for i in list(references['Scratch']['Display name']):
    print(i)
    


theoauzoux
nicolasrihouey
axelhardylainé
arthurtropardy
damienguillemet
nathandouyere
quentinlefebvre
gregorypauchet
olivierpichou
corentincabot
remilejeune
maxencehuard
martinlozier
axelbraghini
arthurdecampeaux
fabiencarpentier
quentinhavé
pierricktravers
hugopegard
alexandreboissout
antoinevottier
julienhuré
collainléo
aurelienarizzi
françoisdumontier
charleslaroche
lucbarbot
alexisdebourges
mathieuchapelle
mathishavard
cyrilhosatte
christianleleu
jimmypigache
tanguydavy
sergedavesne
benoithennequez
richardbarray
sebastienhavard
cedriclegendre
remijouannin
antoninledoux
emmanuelcastus
valentinquibel
paulhaubert
matthieulorgery
vasseurvincent
remigodin
sebastienmarchand
sylvaingoulard
josuepacquet
guillaumeletellier
matheobousser
baptistegallais
pierrebouffay
benoitjehanne
louiscordier
laurentthierry
axellaffont
kentindenis
ériclebotmel
samuelmonier
flaviennourry
corentinangreville
victormangot
fredericfleury
thierryhave
doriandalbart
mathieupitard
guylecomte
julienarriaga
christophe

In [32]:
df = inscrits['Scratch']
for i in list(df['Display name'].apply(clean_club)):
    print(i)

sandrinebourget
emmanuelcastus
nicolasroussel
romaincourtot
aurelienarizzi
aurelienvalle
philippecoral
fredericfleury
blandinecapron
amelieflandrin
alexispoilly
fabiencarpentier
jeremycarpentier
benjaminbreton
regischopart
arnaudbaby
charleslaroche
romaindenommey
samuelpruniaud
anthonydelettre
pierrevallee
élisemauger
thomasboyeldieu
sebastienmarchand
kentindenis
julienmarie
pierricktravers
victorfreullet
remijouannin
maximedenis
benoithennequez
martinlozier
maximelanguet
lucbarbot
richardbarray
mickaelguyot
lalyguyot
ericlebotmel
clemencelejeune
aymericcheron
alexandrescellier
sebastientoufflet
julienhure
francoisdumontier
valentinquibel
christianleleu
josuepacquet
natachabruyez
christophelevillain
loriscapponi
benoitlemaitre
davysehier
ericdore
doniphanjoin
mathisbaudry
arnaudbaudry
mathishavard
sebastienhavard
nilsbion
christophebuchy
olivierpichou
pascalthorel
christopheridel
didierguichaux
patricelebosse
cedriclegendre
arthurdecampeaux
nicolasdiaguili
kylianleberriais
vincentsaint

In [17]:
for i in list(references['Scratch']['Display name']):
    print(i)

theoauzoux
nicolasrihouey
axelhardylainé
arthurtropardy
damienguillemet
nathandouyere
quentinlefebvre
gregorypauchet
olivierpichou
corentincabot
remilejeune
maxencehuard
martinlozier
axelbraghini
arthurdecampeaux
fabiencarpentier
quentinhavé
pierricktravers
hugopegard
alexandreboissout
antoinevottier
julienhuré
collainléo
aurelienarizzi
françoisdumontier
charleslaroche
lucbarbot
alexisdebourges
mathieuchapelle
mathishavard
cyrilhosatte
christianleleu
jimmypigache
tanguydavy
sergedavesne
benoithennequez
richardbarray
sebastienhavard
cedriclegendre
remijouannin
antoninledoux
emmanuelcastus
valentinquibel
paulhaubert
matthieulorgery
vasseurvincent
remigodin
sebastienmarchand
sylvaingoulard
josuepacquet
guillaumeletellier
matheobousser
baptistegallais
pierrebouffay
benoitjehanne
louiscordier
laurentthierry
axellaffont
kentindenis
ériclebotmel
samuelmonier
flaviennourry
corentinangreville
victormangot
fredericfleury
thierryhave
doriandalbart
mathieupitard
guylecomte
julienarriaga
christophe